# ETL for the Snow Emergency Towing Data Sets
The .CSV files for the Snow Emergency data sets differ between snow emergencies.  Columns have differing names, different columns are included in different files, community and neighborhood names are not standard, dates and times are in different formats, etc.  For each file, we will  transform each file to have the following columns:

Date, Time, Location, Latitude, Longitude, Day, Ward, Community, Neighborhood, Emergency.

To achieve this, we will need to:
1.  Rename columns to have uniform names.
2.  Drop extra columns.
3.  Pick a uniform date and time format and split the datetime string into a date and time field.
4.  Pick uniform neighborhood and community names.  The easiest way to do this is to use the City of Minneapolis OpenData GeoJSON files of communities and neighborhoods for uniform names.
5.  Write the final data frame to a `.csv` file to be combined with the other cleaned files.

In [1]:
# import the dependencies
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import geopy

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
# constants related to input and output

DATA_DIR = "data"
OUTPUT_DIR = "output"
WARDS_FILE = "City_Council_Wards.geojson"
COMMUNITIES_FILE = "Communities.geojson"
NEIGHBORHOODS_FILE = "Minneapolis_Neighborhoods.geojson"

In [3]:
import re

input_files = os.listdir(DATA_DIR)

substring_pattern = r'Tows.*csv'
towing_files = [file_name for file_name in input_files if re.search(substring_pattern, file_name)]
towing_files = [file_name for file_name in towing_files if (not ('Grant' in file_name)) and (not ('Polk' in file_name))]
towing_files

['Snow_Emergency_Armatage_Tows_2019.csv',
 'Snow_Emergency_Dana_Tows_2016.csv',
 'Snow_Emergency_Diamond_Lake_Tows_2019.csv',
 'Snow_Emergency_Ferry_Tows_2016.csv',
 'Snow_Emergency_Howe_Tows_2018.csv',
 'Snow_Emergency_Jane_Tows_2017.csv',
 'Snow_Emergency_Olive_Tows_2018.csv',
 'Snow_Emergency_Pembina_Tows_2018.csv',
 'Snow_Emergency_Quincy_Tows_2019.csv',
 'Snow_Emergency_Upton_Tows_2019.csv',
 'Snow_Emergency_Westminster_Tows_2019.csv',
 'Snow_Emergency_Xerxes_Tows_2018.csv',
 'Snow_Emergency_Yale_Tows_2019.csv',
 'Snow_Emergency_Yardville_Tows_2018.csv']

In [4]:
TOWING_FILES = {file_name.split('_')[2]: file_name for file_name in towing_files}
TOWING_FILES

{'Armatage': 'Snow_Emergency_Armatage_Tows_2019.csv',
 'Dana': 'Snow_Emergency_Dana_Tows_2016.csv',
 'Diamond': 'Snow_Emergency_Diamond_Lake_Tows_2019.csv',
 'Ferry': 'Snow_Emergency_Ferry_Tows_2016.csv',
 'Howe': 'Snow_Emergency_Howe_Tows_2018.csv',
 'Jane': 'Snow_Emergency_Jane_Tows_2017.csv',
 'Olive': 'Snow_Emergency_Olive_Tows_2018.csv',
 'Pembina': 'Snow_Emergency_Pembina_Tows_2018.csv',
 'Quincy': 'Snow_Emergency_Quincy_Tows_2019.csv',
 'Upton': 'Snow_Emergency_Upton_Tows_2019.csv',
 'Westminster': 'Snow_Emergency_Westminster_Tows_2019.csv',
 'Xerxes': 'Snow_Emergency_Xerxes_Tows_2018.csv',
 'Yale': 'Snow_Emergency_Yale_Tows_2019.csv',
 'Yardville': 'Snow_Emergency_Yardville_Tows_2018.csv'}

In [5]:
# a helper function to read GeoJSON files in.  GeoPandas read_file can load GeoJSONs

def load_geojson(filename, data_dir=DATA_DIR):
    filepath = os.path.join(data_dir, filename)
    return gpd.read_file(filepath)

def load_csv(filename, data_dir=DATA_DIR):
    filepath = os.path.join(data_dir, filename)
    return pd.read_csv(filepath)

In [6]:
# Look at the first rows of each input data file to see what needs to be adjusted in each case:

def peek_columns(input_files_dict=TOWING_FILES, data_dir=DATA_DIR):
    for key,value in input_files_dict.items():
        tmp = load_csv(value, data_dir)
        print(key + ": ")
        print(tmp.head(1))

In [7]:
peek_columns()

Armatage: 
           X         Y  OBJECTID          Call_Taken      Tow_Reason  \
0 -93.287993  45.00025         1  02/24/2019 09:20pm  Snow Emergency   

                             Location  Latitude  Longitude  Ward   Community  \
0  2101 LYNDALE AVE N ,Minneapolis,MN  45.00025 -93.287993     5  Near North   

  Neighborho  Tow_Zone  Day Snow_Emerg  
0  Hawthorne         1    1   Armatage  
Dana: 
           X          Y  FID          Call_Taken        Customer  \
0 -93.222912  44.901953    1  12/12/2016 03:56am  Snow Emergency   

                       Location    X_Coord    Y_Coord  Ward_numbe  \
0  5555 34 ave s,Minneapolis,MN  44.901953 -93.222912          12   

    Neighborho Community  Day_identi  TowDist  
0  Morris Park   Nokomis           2        6  
Diamond: 
           X          Y  OBJECTID          Call_Taken      Tow_Reason  \
0 -93.247131  44.984731         1  03/10/2019 09:45pm  Snow Emergency   

                       Location   Latitude  Longitude  Ward   Com

# Armatage ETL

In [8]:
armatage = load_csv(TOWING_FILES['Armatage'])
armatage.head()

,X,Y,OBJECTID,Call_Taken,Tow_Reason,Location,Latitude,Longitude,Ward,Community,Neighborho,Tow_Zone,Day,Snow_Emerg
0,-93.287993,45.000250,1,02/24/2019 09:20pm,Snow Emergency,"2101 LYNDALE AVE N ,Minneapolis,MN",45.000250,-93.287993,5,Near North,Hawthorne,1,1,Armatage
1,-93.287993,45.000250,2,02/24/2019 09:23pm,Snow Emergency,"2101 LYNDALE AVE N ,Minneapolis,MN",45.000250,-93.287993,5,Near North,Hawthorne,1,1,Armatage
2,-93.294638,44.951599,3,02/24/2019 09:25pm,Snow Emergency,"2806 EMERSON AVE S,Minneapolis,MN",44.951599,-93.294638,10,Calhoun Isle,Lowry Hill East,3,1,Armatage
3,-93.288395,45.009708,4,02/24/2019 09:35pm,Snow Emergency,"2901 Lyndale Ave n ,Minneapolis,MN",45.009708,-93.288395,5,Near North,Hawthorne,1,1,Armatage
4,-93.294725,44.951578,5,02/24/2019 09:36pm,Snow Emergency,"2800 Emerson Avenue South,Minneapolis,MN",44.951578,-93.294725,10,Calhoun Isle,Lowry Hill East,3,1,Armatage


In [9]:
armatage.rename(columns={'Neighborho': 'Neighborhood', 'Snow_Emerg': 'Emergency'}, inplace=True)

In [10]:
import dateutil.parser as dparser
from datetime import datetime

test = dparser.parse(armatage.loc[0, 'Call_Taken'], fuzzy=True)
print(test.strftime("%m/%d/%Y"))
print(test.strftime("%H:%M"))

02/24/2019
21:20


In [11]:
def get_date(call_string):
    call_dt = dparser.parse(call_string, fuzzy=True)
    return call_dt.strftime("%m/%d/%Y")

def get_time(call_string):
    call_dt = dparser.parse(call_string, fuzzy=True)
    return call_dt.strftime("%H:%M")

armatage['Date'] = armatage.Call_Taken.map(get_date)
armatage['Time'] = armatage.Call_Taken.map(get_time)

In [12]:
# Drop extra columns:
armatage.drop(columns=['X', 'Y', 'OBJECTID', 'Call_Taken', 'Tow_Reason'], inplace=True)
armatage.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Tow_Zone,Day,Emergency,Date,Time
0,"2101 LYNDALE AVE N ,Minneapolis,MN",45.000250,-93.287993,5,Near North,Hawthorne,1,1,Armatage,02/24/2019,21:20
1,"2101 LYNDALE AVE N ,Minneapolis,MN",45.000250,-93.287993,5,Near North,Hawthorne,1,1,Armatage,02/24/2019,21:23
2,"2806 EMERSON AVE S,Minneapolis,MN",44.951599,-93.294638,10,Calhoun Isle,Lowry Hill East,3,1,Armatage,02/24/2019,21:25
3,"2901 Lyndale Ave n ,Minneapolis,MN",45.009708,-93.288395,5,Near North,Hawthorne,1,1,Armatage,02/24/2019,21:35
4,"2800 Emerson Avenue South,Minneapolis,MN",44.951578,-93.294725,10,Calhoun Isle,Lowry Hill East,3,1,Armatage,02/24/2019,21:36


In [13]:
armatage.Community.unique() # These names match the Communities GeoJSON

array(['Near North', 'Calhoun Isle', 'Powderhorn', 'Camden', 'Northeast',
       'Central', 'Southwest', 'Nokomis', 'University', 'Longfellow'],
      dtype=object)

In [14]:
armatage.Neighborhood.unique() # These names do not match the neighborhoods GeoJSON - adopt the neighborhood GeoJSON names.

array(['Hawthorne', 'Lowry Hill East', 'Central', 'McKinley',
       'Waite Park', 'Windom Park', 'Webber - Camden', 'Powderhorn Park',
       'Audubon Park', 'Loring Park', 'Holland', 'North Loop',
       'Linden Hills', 'Page', 'Sheridan', 'Como', 'Bottineau', 'Regina',
       'Hale', 'Bryant', 'Nicollet Island - East Bank', 'Marcy Holmes',
       'Lind - Bohanon', 'Elliot Park', 'Northrop', 'St. Anthony West',
       'East Isles', 'Willard - Hay', 'Windom', 'Longfellow', 'Howe',
       'Folwell', 'Near - North', "Steven's Square - Loring Heights",
       'Sumner - Glenwood', 'Whittier', 'Logan Park', 'Seward',
       'Hiawatha', 'Corcoran', 'Lyndale', 'Lowry Hill', 'ECCO',
       'Bancroft', 'St. Anthony East', 'University of Minnesota',
       'Shingle Creek', 'CARAG', 'Jordan', 'Harrison',
       'Prospect Park - East River Road', 'Cooper', 'King Field',
       'Beltrami', 'Diamond Lake', 'Downtown West', 'Keewaydin',
       'Standish', 'East Harriet', 'Ericsson'], dtype=object)

In [15]:
neighborhoods = load_geojson(NEIGHBORHOODS_FILE)
neighborhoods.head()

,FID,BDNAME,BDNUM,TEXT_NBR,Shape_STAr,Shape_STLe,NCR_LINK,IMAGE,geometry
0,1,Phillips West,90,90,1.066925e+07,14403.885934,http://www.nrp.org/r2/Neighborhoods/Orgs/PHW.html,PHW,"POLYGON ((-93.2625807586419 44.9609082137146, ..."
1,2,Downtown West,87,87,2.075613e+07,19220.602541,http://www.nrp.org/r2/Neighborhoods/Orgs/DTN.html,DTN,"POLYGON ((-93.2601055025157 44.9829952758614, ..."
2,3,Downtown East,88,88,1.025499e+07,13436.601356,http://www.nrp.org/r2/Neighborhoods/Orgs/DTN.html,DTN,"POLYGON ((-93.2449864570206 44.9789336625517, ..."
3,4,Ventura Village,89,89,1.263526e+07,16988.532717,http://www.nrp.org/r2/Neighborhoods/Orgs/VEN.html,VEN,"POLYGON ((-93.24957700344829 44.9662967560422,..."
4,5,Sumner - Glenwood,29,29,5.741860e+06,11065.343364,http://www.nrp.org/r2/Neighborhoods/Orgs/SGL.html,SGL,"POLYGON ((-93.2882976528817 44.9890356035354, ..."


In [16]:
# A helper function to look-up the neigborhood names:

def lookup_neighborhood(latitude, longitude, neighborhood_data_frame):
    point = Point(longitude, latitude)
    
    for row in neighborhood_data_frame.itertuples():
        if row.geometry.contains(point):
            return row.BDNAME
        

def fix_neighborhood_names(df, neighborhood_data_frame):
    nrows = df.shape[0]
    for idx in range(nrows):
        new_neighborhood_name = lookup_neighborhood(df.loc[idx, 'Latitude'], df.loc[idx, 'Longitude'], neighborhood_data_frame)
        if not new_neighborhood_name:
            print(f"{idx}.  Could not find neighborhood.  Old:  {df.Neighborhood[idx]}")
        else:
            old_neighborhood_name = df.loc[idx, 'Neighborhood']
            if old_neighborhood_name != new_neighborhood_name:
                print(f"{idx}.  Old: {old_neighborhood_name} => New: {new_neighborhood_name}")
                df.loc[idx, 'Neighborhood'] = new_neighborhood_name


In [17]:
fix_neighborhood_names(armatage, neighborhoods)

385.  Old: CARAG => New: South Uptown
389.  Old: CARAG => New: South Uptown
424.  Old: CARAG => New: South Uptown
426.  Old: CARAG => New: South Uptown
448.  Old: CARAG => New: South Uptown
452.  Old: CARAG => New: South Uptown
454.  Old: CARAG => New: South Uptown
463.  Old: CARAG => New: South Uptown
474.  Old: CARAG => New: South Uptown
484.  Old: CARAG => New: South Uptown
491.  Old: CARAG => New: South Uptown
516.  Old: CARAG => New: South Uptown
518.  Old: CARAG => New: South Uptown
531.  Old: CARAG => New: South Uptown
540.  Old: CARAG => New: South Uptown
574.  Old: CARAG => New: South Uptown
576.  Old: CARAG => New: South Uptown
578.  Old: CARAG => New: South Uptown
582.  Old: CARAG => New: South Uptown
595.  Old: CARAG => New: South Uptown
633.  Old: CARAG => New: South Uptown
651.  Old: CARAG => New: South Uptown
655.  Old: CARAG => New: South Uptown
672.  Old: CARAG => New: South Uptown
692.  Old: CARAG => New: South Uptown
699.  Old: CARAG => New: South Uptown
720.  Old: C

In [18]:
armatage.Neighborhood.unique()

array(['Hawthorne', 'Lowry Hill East', 'Central', 'McKinley',
       'Waite Park', 'Windom Park', 'Webber - Camden', 'Powderhorn Park',
       'Audubon Park', 'Loring Park', 'Holland', 'North Loop',
       'Linden Hills', 'Page', 'Sheridan', 'Como', 'Bottineau', 'Regina',
       'Hale', 'Bryant', 'Nicollet Island - East Bank', 'Marcy Holmes',
       'Lind - Bohanon', 'Elliot Park', 'Northrop', 'St. Anthony West',
       'East Isles', 'Willard - Hay', 'Windom', 'Longfellow', 'Howe',
       'Folwell', 'Near - North', "Steven's Square - Loring Heights",
       'Sumner - Glenwood', 'Whittier', 'Logan Park', 'Seward',
       'Hiawatha', 'Corcoran', 'Lyndale', 'Lowry Hill', 'ECCO',
       'Bancroft', 'St. Anthony East', 'University of Minnesota',
       'Shingle Creek', 'South Uptown', 'Jordan', 'Harrison',
       'Prospect Park - East River Road', 'Cooper', 'King Field',
       'Beltrami', 'Diamond Lake', 'Downtown West', 'Keewaydin',
       'Standish', 'East Harriet', 'Ericsson'], dtype=ob

In [19]:
armatage.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Tow_Zone,Day,Emergency,Date,Time
0,"2101 LYNDALE AVE N ,Minneapolis,MN",45.000250,-93.287993,5,Near North,Hawthorne,1,1,Armatage,02/24/2019,21:20
1,"2101 LYNDALE AVE N ,Minneapolis,MN",45.000250,-93.287993,5,Near North,Hawthorne,1,1,Armatage,02/24/2019,21:23
2,"2806 EMERSON AVE S,Minneapolis,MN",44.951599,-93.294638,10,Calhoun Isle,Lowry Hill East,3,1,Armatage,02/24/2019,21:25
3,"2901 Lyndale Ave n ,Minneapolis,MN",45.009708,-93.288395,5,Near North,Hawthorne,1,1,Armatage,02/24/2019,21:35
4,"2800 Emerson Avenue South,Minneapolis,MN",44.951578,-93.294725,10,Calhoun Isle,Lowry Hill East,3,1,Armatage,02/24/2019,21:36


In [20]:
armatage.to_csv(os.path.join(OUTPUT_DIR, "Armatage_Tows.csv"), index=False)

# Dana ETL

In [21]:
dana = load_csv(TOWING_FILES['Dana'])
dana.head()

,X,Y,FID,Call_Taken,Customer,Location,X_Coord,Y_Coord,Ward_numbe,Neighborho,Community,Day_identi,TowDist
0,-93.222912,44.901953,1,12/12/2016 03:56am,Snow Emergency,"5555 34 ave s,Minneapolis,MN",44.901953,-93.222912,12,Morris Park,Nokomis,2,6
1,-93.267517,44.969486,2,12/12/2016 03:57am,Snow Emergency,"grant st e / porland ave,Minneapolis,MN",44.969486,-93.267517,7,Elliot Park,Central,2,3
2,-93.278242,44.924943,3,12/12/2016 03:58am,Snow Emergency,"4300 Nicollet ,Minneapolis,MN",44.924943,-93.278242,8,Kingfield,Southwest,2,5
3,-93.242156,44.964005,4,12/12/2016 04:06am,Snow Emergency,"2109 9th st s ,Minneapolis,MN",44.964005,-93.242156,6,Seward,Longfellow,2,4
4,-93.222647,44.902582,5,12/12/2016 04:09am,Snow Emergency,"5535 34th Ave. so,Minneapolis,MN",44.902582,-93.222647,12,Morris Park,Nokomis,2,6


In [22]:
dana.rename(columns={'Neighborho': 'Neighborhood', 'X_Coord': 'Latitude', 'Y_Coord': 'Longitude',\
                     'Ward_numbe': 'Ward', 'Day_identi': 'Day', 'TowDist': 'Tow_Zone'}, inplace=True)
dana.head(1)

,X,Y,FID,Call_Taken,Customer,Location,Latitude,Longitude,Ward,Neighborhood,Community,Day,Tow_Zone
0,-93.222912,44.901953,1,12/12/2016 03:56am,Snow Emergency,"5555 34 ave s,Minneapolis,MN",44.901953,-93.222912,12,Morris Park,Nokomis,2,6


In [23]:
dana['Date'] = dana.Call_Taken.map(get_date)
dana['Time'] = dana.Call_Taken.map(get_time)

In [24]:
dana.drop(columns=['X', 'Y', 'FID', 'Call_Taken', 'Customer'], inplace=True)
dana.head()

,Location,Latitude,Longitude,Ward,Neighborhood,Community,Day,Tow_Zone,Date,Time
0,"5555 34 ave s,Minneapolis,MN",44.901953,-93.222912,12,Morris Park,Nokomis,2,6,12/12/2016,03:56
1,"grant st e / porland ave,Minneapolis,MN",44.969486,-93.267517,7,Elliot Park,Central,2,3,12/12/2016,03:57
2,"4300 Nicollet ,Minneapolis,MN",44.924943,-93.278242,8,Kingfield,Southwest,2,5,12/12/2016,03:58
3,"2109 9th st s ,Minneapolis,MN",44.964005,-93.242156,6,Seward,Longfellow,2,4,12/12/2016,04:06
4,"5535 34th Ave. so,Minneapolis,MN",44.902582,-93.222647,12,Morris Park,Nokomis,2,6,12/12/2016,04:09


In [25]:
dana['Emergency'] = 'Dana'

In [26]:
dana.Community.unique()  # These do not match

array(['Nokomis', 'Central', 'Southwest', 'Longfellow', 'Calhoun-Isles',
       'Near-North', 'University', 'Camden', 'Powderhorn', 'Northeast',
       'Phillips'], dtype=object)

In [27]:
dana.replace({'Calhoun-Isles': 'Calhoun Isle', 'Near-North' : 'Near North'}, inplace=True)
dana.Community.unique()

array(['Nokomis', 'Central', 'Southwest', 'Longfellow', 'Calhoun Isle',
       'Near North', 'University', 'Camden', 'Powderhorn', 'Northeast',
       'Phillips'], dtype=object)

In [28]:
dana.Neighborhood.unique()

array(['Morris Park', 'Elliot Park', 'Kingfield', 'Seward', 'Wenonah',
       'ECCO', 'Hawthorne', 'Marcy-Holmes', 'McKinley', 'Central',
       'Lowry Hill', 'Sumner-Glenwood', 'Lowry Hill East', 'Bancroft',
       'Powderhorn', 'Holland', 'Willard-Hay', 'St. Anthony East',
       'Loring Park', 'Longfellow', 'Near North', 'Whittier', 'Northrop',
       'Ventura Village', 'Audubon Park', 'St. Anthony West', 'Hiawatha',
       'Downtown West', 'Harrison', 'Windom Park', 'Lyndale',
       'Stevens Square', 'Bryant', 'Corcoran', 'Sheridan', 'Lind-Bohanon',
       'Logan Park', 'CARAG', 'Hale', 'East Isles', 'Folwell',
       'North Loop', 'Cooper', 'Nicollet Island', 'Bryn-Mawr', 'Victory',
       'Field', 'Northeast Park', 'Standish', 'Cedar Riverside',
       'Midtown Phillips', 'Regina', 'Webber-Camden', 'Ericsson',
       'East Harriet', 'Como', 'Lynnhurst', 'Jordan', 'Bottineau',
       'Prospect Park', 'Tangletown', 'Keewaydin', 'Howe', 'University',
       'Waite Park', 'Marshall 

In [29]:
fix_neighborhood_names(dana, neighborhoods)

2.  Old: Kingfield => New: King Field
14.  Old: Marcy-Holmes => New: Marcy Holmes
17.  Old: Marcy-Holmes => New: Marcy Holmes
19.  Old: Marcy-Holmes => New: Marcy Holmes
23.  Old: Sumner-Glenwood => New: Sumner - Glenwood
26.  Old: Sumner-Glenwood => New: Sumner - Glenwood
30.  Old: Sumner-Glenwood => New: Sumner - Glenwood
34.  Old: Sumner-Glenwood => New: Sumner - Glenwood
36.  Old: Sumner-Glenwood => New: Sumner - Glenwood
39.  Old: Powderhorn => New: Powderhorn Park
40.  Old: Sumner-Glenwood => New: Sumner - Glenwood
41.  Old: Powderhorn => New: Powderhorn Park
43.  Old: Marcy-Holmes => New: Marcy Holmes
46.  Old: Willard-Hay => New: Willard - Hay
47.  Old: Marcy-Holmes => New: Marcy Holmes
50.  Old: Willard-Hay => New: Willard - Hay
52.  Old: Powderhorn => New: Powderhorn Park
54.  Old: Kingfield => New: King Field
59.  Old: Near North => New: Near - North
60.  Old: Marcy-Holmes => New: Marcy Holmes
66.  Old: Sumner-Glenwood => New: Sumner - Glenwood
70.  Old: Marcy-Holmes => New:

554.  Old: Marcy-Holmes => New: Marcy Holmes
557.  Old: Marcy-Holmes => New: Marcy Holmes
561.  Old: Marcy-Holmes => New: Marcy Holmes
563.  Old: Marcy-Holmes => New: Marcy Holmes
564.  Old: Sumner-Glenwood => New: Sumner - Glenwood
566.  Old: Marcy-Holmes => New: Marcy Holmes
567.  Old: Stevens Square => New: Steven's Square - Loring Heights
569.  Old: Marcy-Holmes => New: Marcy Holmes
570.  Old: Marcy-Holmes => New: Marcy Holmes
571.  Old: Stevens Square => New: Steven's Square - Loring Heights
575.  Old: Near North => New: Near - North
576.  Old: Stevens Square => New: Steven's Square - Loring Heights
580.  Old: Near North => New: Near - North
581.  Old: Marcy-Holmes => New: Marcy Holmes
583.  Old: Marcy-Holmes => New: Marcy Holmes
584.  Old: Near North => New: Near - North
585.  Old: Near North => New: Near - North
587.  Old: Stevens Square => New: Steven's Square - Loring Heights
590.  Old: Stevens Square => New: Steven's Square - Loring Heights
593.  Old: Stevens Square => New: S

1008.  Old: Willard-Hay => New: Willard - Hay
1012.  Old: Lind-Bohanon => New: Lind - Bohanon
1015.  Old: Kingfield => New: King Field
1019.  Old: Marcy-Holmes => New: Marcy Holmes
1023.  Old: Near North => New: Near - North
1024.  Old: Marcy-Holmes => New: Marcy Holmes
1025.  Old: Marcy-Holmes => New: Marcy Holmes
1029.  Old: Marcy-Holmes => New: Marcy Holmes
1030.  Old: Marcy-Holmes => New: Marcy Holmes
1031.  Old: Marcy-Holmes => New: Marcy Holmes
1032.  Old: Marcy-Holmes => New: Marcy Holmes
1034.  Old: Marcy-Holmes => New: Marcy Holmes
1037.  Old: Sumner-Glenwood => New: Sumner - Glenwood
1038.  Old: Near North => New: Near - North
1040.  Old: Near North => New: Near - North
1041.  Old: Sumner-Glenwood => New: Sumner - Glenwood
1042.  Old: Sumner-Glenwood => New: Sumner - Glenwood
1053.  Old: Marcy-Holmes => New: Marcy Holmes
1054.  Old: Webber-Camden => New: Webber - Camden
1055.  Old: Near North => New: Near - North
1056.  Old: CARAG => New: South Uptown
1057.  Old: Willard-Hay 

In [30]:
dana.Neighborhood.unique()

array(['Morris Park', 'Elliot Park', 'King Field', 'Seward', 'Wenonah',
       'ECCO', 'Hawthorne', 'Marcy Holmes', 'McKinley', 'Central',
       'Lowry Hill', 'Sumner - Glenwood', 'Lowry Hill East', 'Bancroft',
       'Powderhorn Park', 'Holland', 'Willard - Hay', 'St. Anthony East',
       'Loring Park', 'Longfellow', 'Near - North', 'Whittier',
       'Northrop', 'Ventura Village', 'Audubon Park', 'St. Anthony West',
       'Hiawatha', 'Downtown West', 'Harrison', 'Windom Park', 'Lyndale',
       "Steven's Square - Loring Heights", 'Bryant', 'Corcoran',
       'Sheridan', 'Lind - Bohanon', 'Logan Park', 'South Uptown', 'Hale',
       'East Isles', 'Folwell', 'North Loop', 'Cooper',
       'Nicollet Island - East Bank', 'Bryn - Mawr', 'Victory', 'Field',
       'Northeast Park', 'Standish', 'Cedar Riverside',
       'Midtown Phillips', 'Regina', 'Webber - Camden', 'Ericsson',
       'East Harriet', 'Como', 'Lynnhurst', 'Jordan', 'Bottineau',
       'Prospect Park - East River Road', 

In [31]:
dana[dana.Neighborhood.isnull()]

,Location,Latitude,Longitude,Ward,Neighborhood,Community,Day,Tow_Zone,Date,Time,Emergency


In [32]:
dana.head()

,Location,Latitude,Longitude,Ward,Neighborhood,Community,Day,Tow_Zone,Date,Time,Emergency
0,"5555 34 ave s,Minneapolis,MN",44.901953,-93.222912,12,Morris Park,Nokomis,2,6,12/12/2016,03:56,Dana
1,"grant st e / porland ave,Minneapolis,MN",44.969486,-93.267517,7,Elliot Park,Central,2,3,12/12/2016,03:57,Dana
2,"4300 Nicollet ,Minneapolis,MN",44.924943,-93.278242,8,King Field,Southwest,2,5,12/12/2016,03:58,Dana
3,"2109 9th st s ,Minneapolis,MN",44.964005,-93.242156,6,Seward,Longfellow,2,4,12/12/2016,04:06,Dana
4,"5535 34th Ave. so,Minneapolis,MN",44.902582,-93.222647,12,Morris Park,Nokomis,2,6,12/12/2016,04:09,Dana


In [34]:
dana.to_csv(os.path.join(OUTPUT_DIR, "Dana_Tows.csv"), index=False)

# Diamond Lake

In [35]:
diamond = load_csv(TOWING_FILES['Diamond'])
diamond.head()

,X,Y,OBJECTID,Call_Taken,Tow_Reason,Location,Latitude,Longitude,Ward,Community,Neighborho,Tow_Zone,Day,Snow_Emerg
0,-93.247131,44.984731,1,03/10/2019 09:45pm,Snow Emergency,"592 4TH ST SE,Minneapolis,MN",44.984731,-93.247131,3,University,Marcy Holmes,4,1,Diamond Lake
1,-93.278567,44.967720,2,03/10/2019 09:45pm,Snow Emergency,"w 15th/ oak grove,Minneapolis,MN",44.967720,-93.278567,7,Central,Loring Park,3,1,Diamond Lake
2,-93.232459,44.933124,3,03/10/2019 09:47pm,Snow Emergency,"3832 28th Avenue s,Minneapolis,MN",44.933124,-93.232459,12,Powderhorn,Standish,6,1,Diamond Lake
3,-93.248586,44.985306,4,03/10/2019 09:51pm,Snow Emergency,"464 4TH ST SE,Minneapolis,MN",44.985306,-93.248586,3,University,Marcy Holmes,4,1,Diamond Lake
4,-93.287993,45.000250,5,03/10/2019 09:54pm,Snow Emergency,"2101-2199 lyndale ave n ,Minneapolis,MN",45.000250,-93.287993,5,Near North,Hawthorne,1,1,Diamond Lake


In [36]:
diamond.rename(columns={'Neighborho': 'Neighborhood', 'Snow_Emerg': 'Emergency'}, inplace=True)

In [37]:
diamond['Date'] = diamond.Call_Taken.map(get_date)
diamond['Time'] = diamond.Call_Taken.map(get_time)

In [38]:
diamond.drop(columns=['X', 'Y', 'OBJECTID', 'Call_Taken', 'Tow_Reason'], inplace=True)
diamond.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Tow_Zone,Day,Emergency,Date,Time
0,"592 4TH ST SE,Minneapolis,MN",44.984731,-93.247131,3,University,Marcy Holmes,4,1,Diamond Lake,03/10/2019,21:45
1,"w 15th/ oak grove,Minneapolis,MN",44.967720,-93.278567,7,Central,Loring Park,3,1,Diamond Lake,03/10/2019,21:45
2,"3832 28th Avenue s,Minneapolis,MN",44.933124,-93.232459,12,Powderhorn,Standish,6,1,Diamond Lake,03/10/2019,21:47
3,"464 4TH ST SE,Minneapolis,MN",44.985306,-93.248586,3,University,Marcy Holmes,4,1,Diamond Lake,03/10/2019,21:51
4,"2101-2199 lyndale ave n ,Minneapolis,MN",45.000250,-93.287993,5,Near North,Hawthorne,1,1,Diamond Lake,03/10/2019,21:54


In [39]:
diamond.Community.unique()

array(['University', 'Central', 'Powderhorn', 'Near North', 'Nokomis',
       'Northeast', 'Camden', 'Calhoun Isle', 'Southwest', 'Phillips',
       'Longfellow'], dtype=object)

In [40]:
fix_neighborhood_names(diamond, neighborhoods)

240.  Old: CARAG => New: South Uptown
246.  Old: CARAG => New: South Uptown
375.  Old: CARAG => New: South Uptown
393.  Old: CARAG => New: South Uptown
406.  Old: CARAG => New: South Uptown
409.  Old: CARAG => New: South Uptown
412.  Old: CARAG => New: South Uptown
423.  Old: CARAG => New: South Uptown
426.  Old: CARAG => New: South Uptown
434.  Old: CARAG => New: South Uptown
452.  Old: CARAG => New: South Uptown
519.  Old: CARAG => New: South Uptown


In [41]:
diamond.to_csv(os.path.join(OUTPUT_DIR, "Diamond_Tows.csv"), index=False)

# Ferry 2016

In [42]:
ferry = load_csv(TOWING_FILES['Ferry'])
ferry.head()

,X,Y,FID,Call_Taken,Customer,Location,X_Coord,Y_Coord,Ward_numbe,Neighborho,Community,Day_identi,TowDist
0,-93.280233,45.005508,1,12/17/2016 09:15pm,Snow Emergency,"2563 2nd st n. ,Minneapolis,MN",45.005508,-93.280233,5,Hawthorne,Near-North,1,1
1,-93.263027,44.947734,2,12/17/2016 09:21pm,Snow Emergency,"3018 Chicago Avenue S,Minneapolis,MN",44.947734,-93.263027,9,Central,Powderhorn,1,5
2,-93.262996,44.943660,3,12/17/2016 09:23pm,Snow Emergency,"3232 Chicago ave,Minneapolis,MN",44.943660,-93.262996,9,Central,Powderhorn,1,5
3,-93.294470,44.951574,4,12/17/2016 09:24pm,Snow Emergency,"2802 EMERSON AVE S,Minneapolis,MN",44.951574,-93.294470,10,Lowry Hill East,Calhoun Isles,1,3
4,-93.294471,44.951547,5,12/17/2016 09:26pm,Snow Emergency,"2814 EMERSON AVE S,Minneapolis,MN",44.951547,-93.294471,10,Lowry Hill East,Calhoun Isles,1,3


In [43]:
ferry.rename(columns={'Neighborho': 'Neighborhood', 'X_Coord': 'Latitude', 'Y_Coord': 'Longitude',\
                     'Ward_numbe': 'Ward', 'Day_identi': 'Day', 'TowDist': 'Tow_Zone'}, inplace=True)

ferry['Date'] = ferry.Call_Taken.map(get_date)
ferry['Time'] = ferry.Call_Taken.map(get_time)
ferry.drop(columns=['X', 'Y', 'FID', 'Call_Taken', 'Customer'], inplace=True)
ferry['Emergency'] = 'Ferry'

ferry.head()

,Location,Latitude,Longitude,Ward,Neighborhood,Community,Day,Tow_Zone,Date,Time,Emergency
0,"2563 2nd st n. ,Minneapolis,MN",45.005508,-93.280233,5,Hawthorne,Near-North,1,1,12/17/2016,21:15,Ferry
1,"3018 Chicago Avenue S,Minneapolis,MN",44.947734,-93.263027,9,Central,Powderhorn,1,5,12/17/2016,21:21,Ferry
2,"3232 Chicago ave,Minneapolis,MN",44.943660,-93.262996,9,Central,Powderhorn,1,5,12/17/2016,21:23,Ferry
3,"2802 EMERSON AVE S,Minneapolis,MN",44.951574,-93.294470,10,Lowry Hill East,Calhoun Isles,1,3,12/17/2016,21:24,Ferry
4,"2814 EMERSON AVE S,Minneapolis,MN",44.951547,-93.294471,10,Lowry Hill East,Calhoun Isles,1,3,12/17/2016,21:26,Ferry


In [44]:
ferry.Community.unique()

array(['Near-North', 'Powderhorn', 'Calhoun Isles', 'Nokomis', 'Central',
       'Northeast', 'University', 'Camden', 'Phillips', 'Longfellow',
       'Southwest'], dtype=object)

In [45]:
ferry.replace({'Calhoun-Isles': 'Calhoun Isle', 'Near-North' : 'Near North'}, inplace=True)
ferry.Community.unique()

array(['Near North', 'Powderhorn', 'Calhoun Isles', 'Nokomis', 'Central',
       'Northeast', 'University', 'Camden', 'Phillips', 'Longfellow',
       'Southwest'], dtype=object)

In [46]:
fix_neighborhood_names(ferry, neighborhoods)

17.  Old: Marcy-Holmes => New: Marcy Holmes
19.  Old: Near North => New: Near - North
23.  Old: CARAG => New: South Uptown
24.  Old: Marcy-Holmes => New: Marcy Holmes
26.  Old: CARAG => New: South Uptown
31.  Old: Marcy-Holmes => New: Marcy Holmes
33.  Old: Stevens Square => New: Steven's Square - Loring Heights
39.  Old: Near North => New: Near - North
40.  Old: Willard-Hay => New: Willard - Hay
41.  Old: Marcy-Holmes => New: Marcy Holmes
48.  Old: Marcy-Holmes => New: Marcy Holmes
52.  Old: Marcy-Holmes => New: Marcy Holmes
53.  Old: Webber-Camden => New: Webber - Camden
60.  Old: Near North => New: Near - North
62.  Old: Willard-Hay => New: Willard - Hay
63.  Old: CARAG => New: South Uptown
66.  Old: Stevens Square => New: Steven's Square - Loring Heights
75.  Old: Marcy-Holmes => New: Marcy Holmes
78.  Old: Stevens Square => New: Steven's Square - Loring Heights
83.  Old: Marcy-Holmes => New: Marcy Holmes
85.  Old: Willard-Hay => New: Willard - Hay
88.  Old: Marcy-Holmes => New: Ma

559.  Old: Near North => New: Near - North
562.  Old: Prospect Park => New: Prospect Park - East River Road
570.  Old: Marcy-Holmes => New: Marcy Holmes
571.  Old: Marcy-Holmes => New: Marcy Holmes
573.  Old: CARAG => New: South Uptown
574.  Old: Marcy-Holmes => New: Marcy Holmes
575.  Old: Sumner-Glenwood => New: Sumner - Glenwood
578.  Old: Stevens Square => New: Steven's Square - Loring Heights
579.  Old: Near North => New: Near - North
580.  Old: CARAG => New: South Uptown
582.  Old: Willard-Hay => New: Willard - Hay
583.  Old: Near North => New: Near - North
588.  Old: Sumner-Glenwood => New: Sumner - Glenwood
589.  Old: Marcy-Holmes => New: Marcy Holmes
590.  Old: Marcy-Holmes => New: Marcy Holmes
592.  Old: Marcy-Holmes => New: Marcy Holmes
593.  Old: Marcy-Holmes => New: Marcy Holmes
595.  Old: Marcy-Holmes => New: Marcy Holmes
596.  Old: University => New: University of Minnesota
597.  Old: Sumner-Glenwood => New: Sumner - Glenwood
603.  Old: Marcy-Holmes => New: Marcy Holmes


1098.  Old: CARAG => New: South Uptown
1100.  Old: Marcy-Holmes => New: Marcy Holmes
1101.  Old: Marcy-Holmes => New: Marcy Holmes
1102.  Old: Marcy-Holmes => New: Marcy Holmes
1104.  Old: Stevens Square => New: Steven's Square - Loring Heights
1107.  Old: Near North => New: Near - North
1108.  Old: Marcy-Holmes => New: Marcy Holmes
1109.  Old: Marcy-Holmes => New: Marcy Holmes
1110.  Old: Marcy-Holmes => New: Marcy Holmes
1111.  Old: Near North => New: Near - North
1115.  Old: Stevens Square => New: Steven's Square - Loring Heights
1120.  Old: Near North => New: Near - North
1126.  Old: Near North => New: Near - North
1131.  Old: Marcy-Holmes => New: Marcy Holmes
1132.  Old: Marcy-Holmes => New: Marcy Holmes
1133.  Old: Willard-Hay => New: Willard - Hay
1140.  Old: Willard-Hay => New: Willard - Hay
1143.  Old: Marcy-Holmes => New: Marcy Holmes
1145.  Old: Marcy-Holmes => New: Marcy Holmes
1148.  Could not find neighborhood.  Old:  Waite Park
1150.  Old: Near North => New: Near - North

In [47]:
ferry.Neighborhood.isnull().sum()

0

In [48]:
ferry.to_csv(os.path.join(OUTPUT_DIR, "Ferry_Tows.csv"), index=False)

# Howe 2018

In [51]:
howe=load_csv(TOWING_FILES['Howe'])
howe.head()

,X,Y,FID,Service_Da,Address,Tow_Reason,Latitude,Longitude,Ward,Community,Neighborho,Tow_Zone,Day,Snow_Emerg
0,-93.310768,44.985471,1,04/16/2018 02:43pm,"710 RUSSELL AVE N,Minneapolis,MN",Snow Emergency,44.985471,-93.310768,5,Near North,Willard - Hay,1,2,Howe
1,-93.220151,44.971199,2,04/16/2018 02:43pm,"150 26th Ave SE,Minneapolis,MN",Snow Emergency,44.971199,-93.220151,2,University,Prospect Park - East River Road,4,2,Howe
2,-93.262426,45.006359,3,04/16/2018 02:44pm,"360 18th Ave NE,Minneapolis,MN",Snow Emergency,45.006359,-93.262426,1,Northeast,Holland,2,2,Howe
3,-93.227075,44.939345,4,04/16/2018 02:46pm,"3520 31st st e ,Minneapolis,MN",Snow Emergency,44.939345,-93.227075,12,Longfellow,Howe,6,2,Howe
4,-93.294223,45.027867,5,04/16/2018 02:47pm,"4004 Emerson Ave n,Minneapolis,MN",Snow Emergency,45.027867,-93.294223,4,Camden,Webber - Camden,1,2,Howe


In [52]:
# Based on the above, we can follow the diamond lake cleaning:
howe.rename(columns={'Address': 'Location', 'Neighborho': 'Neighborhood', 'Snow_Emerg': 'Emergency'}, inplace=True)

howe['Date'] = howe.Service_Da.map(get_date)
howe['Time'] = howe.Service_Da.map(get_time)

howe.drop(columns=['X', 'Y', 'FID', 'Service_Da', 'Tow_Reason'], inplace=True)
howe.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Tow_Zone,Day,Emergency,Date,Time
0,"710 RUSSELL AVE N,Minneapolis,MN",44.985471,-93.310768,5,Near North,Willard - Hay,1,2,Howe,04/16/2018,14:43
1,"150 26th Ave SE,Minneapolis,MN",44.971199,-93.220151,2,University,Prospect Park - East River Road,4,2,Howe,04/16/2018,14:43
2,"360 18th Ave NE,Minneapolis,MN",45.006359,-93.262426,1,Northeast,Holland,2,2,Howe,04/16/2018,14:44
3,"3520 31st st e ,Minneapolis,MN",44.939345,-93.227075,12,Longfellow,Howe,6,2,Howe,04/16/2018,14:46
4,"4004 Emerson Ave n,Minneapolis,MN",45.027867,-93.294223,4,Camden,Webber - Camden,1,2,Howe,04/16/2018,14:47


In [53]:
howe.Community.unique() # these are ok

array(['Near North', 'University', 'Northeast', 'Longfellow', 'Camden',
       'Powderhorn', 'Central', 'Calhoun Isle', 'Nokomis', 'Southwest',
       'Phillips'], dtype=object)

In [54]:
fix_neighborhood_names(howe, neighborhoods)

25.  Old: CARAG => New: South Uptown
28.  Old: CARAG => New: South Uptown
135.  Old: CARAG => New: South Uptown
140.  Old: CARAG => New: South Uptown
144.  Old: CARAG => New: South Uptown
146.  Old: CARAG => New: South Uptown
147.  Old: CARAG => New: South Uptown
164.  Old: CARAG => New: South Uptown
168.  Old: CARAG => New: South Uptown
181.  Old: CARAG => New: South Uptown
190.  Old: CARAG => New: South Uptown
223.  Old: CARAG => New: South Uptown
243.  Old: CARAG => New: South Uptown
245.  Old: CARAG => New: South Uptown
250.  Old: CARAG => New: South Uptown
277.  Old: CARAG => New: South Uptown
278.  Old: CARAG => New: South Uptown
289.  Old: CARAG => New: South Uptown
292.  Old: CARAG => New: South Uptown
313.  Old: CARAG => New: South Uptown
356.  Old: CARAG => New: South Uptown
469.  Old: CARAG => New: South Uptown
588.  Old: CARAG => New: South Uptown
689.  Old: CARAG => New: South Uptown
722.  Old: CARAG => New: South Uptown
728.  Old: CARAG => New: South Uptown
746.  Old: CAR

In [55]:
howe.to_csv(os.path.join(OUTPUT_DIR, "Howe_Tows.csv"), index=False)

# Jane 2017

In [66]:
jane = load_csv(TOWING_FILES['Jane'])
jane.head()

,X,Y,OBJECTID,Call_Taken,Customer,Location,Latitude,Longitude,Ward_numbe,Neighborho,Community,Day_Identi,TowDist
0,-93.217582,44.973232,1001,01/13/2017 02:10pm,Snow Emergency,"2709 4th St se,Minneapolis,MN",44.973232,-93.217582,2,Prospect Park,University,3,4
1,-93.216956,44.973393,1002,01/13/2017 02:13pm,Snow Emergency,"2715 4thSST se,Minneapolis,MN",44.973393,-93.216956,2,Prospect Park,University,3,4
2,-93.216539,44.972820,1003,01/13/2017 02:14pm,Snow Emergency,"2807 4th st se,Minneapolis,MN",44.972820,-93.216539,2,Prospect Park,University,3,4
3,-93.221945,44.972383,1004,01/13/2017 02:16pm,Snow Emergency,"2513 Delaware st se,Minneapolis,MN",44.972383,-93.221945,2,Prospect Park,University,3,4
4,-93.220766,44.971919,1005,01/13/2017 02:19pm,Snow Emergency,"2601 Delaware st se,Minneapolis,MN",44.971919,-93.220766,2,Prospect Park,University,3,4


In [67]:
# Based on the file, this will follow the methods used for Ferry:

jane.rename(columns={'Neighborho': 'Neighborhood',\
                     'Ward_numbe': 'Ward', 'Day_Identi': 'Day', 'TowDist': 'Tow_Zone'}, inplace=True)

jane['Date'] = jane.Call_Taken.map(get_date)
jane['Time'] = jane.Call_Taken.map(get_time)
jane.drop(columns=['X', 'Y', 'OBJECTID', 'Call_Taken', 'Customer'], inplace=True)
jane['Emergency'] = 'Jane'

jane.head()

,Location,Latitude,Longitude,Ward,Neighborhood,Community,Day,Tow_Zone,Date,Time,Emergency
0,"2709 4th St se,Minneapolis,MN",44.973232,-93.217582,2,Prospect Park,University,3,4,01/13/2017,14:10,Jane
1,"2715 4thSST se,Minneapolis,MN",44.973393,-93.216956,2,Prospect Park,University,3,4,01/13/2017,14:13,Jane
2,"2807 4th st se,Minneapolis,MN",44.972820,-93.216539,2,Prospect Park,University,3,4,01/13/2017,14:14,Jane
3,"2513 Delaware st se,Minneapolis,MN",44.972383,-93.221945,2,Prospect Park,University,3,4,01/13/2017,14:16,Jane
4,"2601 Delaware st se,Minneapolis,MN",44.971919,-93.220766,2,Prospect Park,University,3,4,01/13/2017,14:19,Jane


In [68]:
jane.Community.unique()

array(['University', 'Nokomis', 'Longfellow', 'Northeast', 'Powderhorn',
       'Central', 'Near North', 'Phillips', 'Camden', 'Calhoun Isles',
       'Southwest'], dtype=object)

In [69]:
jane.replace({'Calhoun Isles': 'Calhoun Isle'}, inplace=True)

In [70]:
jane.Community.unique()

array(['University', 'Nokomis', 'Longfellow', 'Northeast', 'Powderhorn',
       'Central', 'Near North', 'Phillips', 'Camden', 'Calhoun Isle',
       'Southwest'], dtype=object)

In [71]:
fix_neighborhood_names(jane, neighborhoods)

0.  Old: Prospect Park => New: Prospect Park - East River Road
1.  Old: Prospect Park => New: Prospect Park - East River Road
2.  Old: Prospect Park => New: Prospect Park - East River Road
3.  Old: Prospect Park => New: Prospect Park - East River Road
4.  Old: Prospect Park => New: Prospect Park - East River Road
5.  Old: Prospect Park => New: Prospect Park - East River Road
6.  Old: Prospect Park => New: Prospect Park - East River Road
130.  Old: Stevens Square => New: Steven's Square - Loring Heights
131.  Old: Stevens Square => New: Steven's Square - Loring Heights
132.  Old: Stevens Square => New: Steven's Square - Loring Heights
133.  Old: Stevens Square => New: Steven's Square - Loring Heights
134.  Old: Stevens Square => New: Steven's Square - Loring Heights
135.  Old: Stevens Square => New: Steven's Square - Loring Heights
136.  Old: Stevens Square => New: Steven's Square - Loring Heights
137.  Old: Stevens Square => New: Steven's Square - Loring Heights
138.  Old: Stevens Squa

470.  Old: CARAG => New: South Uptown
471.  Old: CARAG => New: South Uptown
472.  Old: CARAG => New: South Uptown
473.  Old: CARAG => New: South Uptown
474.  Old: CARAG => New: South Uptown
475.  Old: CARAG => New: South Uptown
476.  Old: CARAG => New: South Uptown
477.  Old: CARAG => New: South Uptown
478.  Old: CARAG => New: South Uptown
479.  Old: CARAG => New: South Uptown
480.  Old: CARAG => New: South Uptown
481.  Old: CARAG => New: South Uptown
482.  Old: CARAG => New: South Uptown
483.  Old: CARAG => New: South Uptown
484.  Old: CARAG => New: South Uptown
485.  Old: CARAG => New: South Uptown
486.  Old: Cedar-Riverside => New: Cedar Riverside
487.  Old: Cedar-Riverside => New: Cedar Riverside
488.  Old: Cedar-Riverside => New: Cedar Riverside
489.  Old: Cedar-Riverside => New: Cedar Riverside
490.  Old: Cedar-Riverside => New: Cedar Riverside
491.  Old: Cedar-Riverside => New: Cedar Riverside
492.  Old: Cedar-Riverside => New: Cedar Riverside
493.  Old: Cedar-Riverside => New: 

In [72]:
jane.Neighborhood.isnull().sum()

0

In [73]:
jane.to_csv(os.path.join(OUTPUT_DIR, "Jane_Tows.csv"), index=False)

# Olive 2018

In [83]:
olive = load_csv(TOWING_FILES['Olive'])
olive.head()

,X,Y,OBJECTID,Call_Taken,Customer,Location,Latitude,Longitude,Ward,Community,Neighborho,Day_identi,TowDist
0,-93.300925,44.992478,1001,01/17/2018 01:20pm,Snow Emergency,"1329 JAMES AVE N,Minneapolis,MN",44.992478,-93.300925,5,Near North,Near - North,3,1
1,-93.278752,44.966374,1002,01/17/2018 12:12pm,Snow Emergency,"15 OAK GROVE STREET,Minneapolis,MN",44.966374,-93.278752,7,Central,Loring Park,3,3
2,-93.285889,44.967008,1003,01/17/2018 01:21pm,Snow Emergency,"1723 CLIFTON PL,Minneapolis,MN",44.967008,-93.285889,7,Central,Loring Park,3,3
3,-93.283318,44.965572,1004,01/17/2018 01:22pm,Snow Emergency,"245 CLIFTON AVE,Minneapolis,MN",44.965572,-93.283318,7,Central,Loring Park,3,3
4,-93.297076,44.942203,1005,01/17/2018 01:23pm,Snow Emergency,"3323 GIRARD AVE S,Minneapolis,MN",44.942203,-93.297076,10,Calhoun Isle,CARAG,3,5


In [84]:
# Based on the file, this will follow the methods used for Ferry:

olive.rename(columns={'Neighborho': 'Neighborhood',\
                     'Day_identi': 'Day', 'TowDist': 'Tow_Zone'}, inplace=True)

olive['Date'] = olive.Call_Taken.map(get_date)
olive['Time'] = olive.Call_Taken.map(get_time)
olive.drop(columns=['X', 'Y', 'OBJECTID', 'Call_Taken', 'Customer'], inplace=True)
olive['Emergency'] = 'Olive'

olive.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Day,Tow_Zone,Date,Time,Emergency
0,"1329 JAMES AVE N,Minneapolis,MN",44.992478,-93.300925,5,Near North,Near - North,3,1,01/17/2018,13:20,Olive
1,"15 OAK GROVE STREET,Minneapolis,MN",44.966374,-93.278752,7,Central,Loring Park,3,3,01/17/2018,12:12,Olive
2,"1723 CLIFTON PL,Minneapolis,MN",44.967008,-93.285889,7,Central,Loring Park,3,3,01/17/2018,13:21,Olive
3,"245 CLIFTON AVE,Minneapolis,MN",44.965572,-93.283318,7,Central,Loring Park,3,3,01/17/2018,13:22,Olive
4,"3323 GIRARD AVE S,Minneapolis,MN",44.942203,-93.297076,10,Calhoun Isle,CARAG,3,5,01/17/2018,13:23,Olive


In [85]:
olive.Community.unique()

array(['Near North', 'Central', 'Calhoun Isle', 'University',
       'Powderhorn', 'Northeast', 'Camden', 'Nokomis', 'Longfellow',
       'Phillips', 'Southwest'], dtype=object)

In [86]:
fix_neighborhood_names(olive, neighborhoods)

4.  Old: CARAG => New: South Uptown
5.  Old: Marcy - Holmes => New: Marcy Holmes
9.  Old: CARAG => New: South Uptown
11.  Old: Marcy - Holmes => New: Marcy Holmes
14.  Old: CARAG => New: South Uptown
20.  Old: Marcy - Holmes => New: Marcy Holmes
21.  Old: Marcy - Holmes => New: Marcy Holmes
25.  Old: Marcy - Holmes => New: Marcy Holmes
27.  Old: Marcy - Holmes => New: Marcy Holmes
29.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
30.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
35.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
36.  Old: Marcy - Holmes => New: Marcy Holmes
38.  Old: Marcy - Holmes => New: Marcy Holmes
40.  Old: Marcy - Holmes => New: Marcy Holmes
42.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
46.  Old: Marcy - Holmes => New: Marcy Holmes
48.  Old: Marcy - Holmes => New: Marcy Holmes
49.  Old: CARAG => New: South Uptown
51.  Old: Marcy -

842.  Old: Marcy - Holmes => New: Marcy Holmes
843.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
846.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
855.  Old: Marcy - Holmes => New: Marcy Holmes
857.  Old: Marcy - Holmes => New: Marcy Holmes
861.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
866.  Old: Marcy - Holmes => New: Marcy Holmes
879.  Old: Marcy - Holmes => New: Marcy Holmes
881.  Old: Marcy - Holmes => New: Marcy Holmes
885.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
886.  Old: Marcy - Holmes => New: Marcy Holmes
889.  Old: Northrup => New: Northrop
891.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
893.  Old: Marcy - Holmes => New: Marcy Holmes
898.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights
899.  Old: Stevens Square - Loring Heights => New: Steven's Square - Loring Heights

In [87]:
olive.Neighborhood.isnull().sum()

0

In [88]:
olive.to_csv(os.path.join(OUTPUT_DIR, "Olive_Tows.csv"), index=False)

# Pembina 2018

In [89]:
pembina = load_csv(TOWING_FILES['Pembina'])
pembina.head()

,X,Y,OBJECTID,Call_Taken,Customer,Location,Latitude,Longitude,Day,Ward,Neighborho,Community,TowDist
0,-93.281105,45.014320,1,1/22/2018 9:26:00pm,Snow Emergency,"3235 2nd st n ,Minneapolis,MN",45.014320,-93.281105,1,4,McKinley,Camden,1
1,-93.287688,45.039590,2,1/22/2018 9:30:00pm,Snow Emergency,"4635 lyndale ave n,Minneapolis,MN",45.039590,-93.287688,1,4,Lind - Bohanon,Camden,1
2,-93.279508,45.004469,3,1/22/2018 9:35:00pm,Snow Emergency,"2430 2nd St n ,Minneapolis,MN",45.004469,-93.279508,1,5,Hawthorne,Near North,1
3,-93.287739,45.039000,4,1/22/2018 9:38:00pm,Snow Emergency,"4615 lyndale ave n,Minneapolis,MN",45.039000,-93.287739,1,4,Lind - Bohanon,Camden,1
4,-93.262878,44.938743,5,1/22/2018 9:43:00pm,Snow Emergency,"3520 Chicago ave s,Minneapolis,MN",44.938743,-93.262878,1,9,Central,Powderhorn,5


In [90]:
pembina.rename(columns={'Neighborho': 'Neighborhood', 'TowDist': 'Tow_Zone'}, inplace=True)

pembina['Date'] = pembina.Call_Taken.map(get_date)
pembina['Time'] = pembina.Call_Taken.map(get_time)
pembina.drop(columns=['X', 'Y', 'OBJECTID', 'Call_Taken', 'Customer'], inplace=True)
pembina['Emergency'] = 'Pembina'

pembina.head()

,Location,Latitude,Longitude,Day,Ward,Neighborhood,Community,Tow_Zone,Date,Time,Emergency
0,"3235 2nd st n ,Minneapolis,MN",45.014320,-93.281105,1,4,McKinley,Camden,1,01/22/2018,21:26,Pembina
1,"4635 lyndale ave n,Minneapolis,MN",45.039590,-93.287688,1,4,Lind - Bohanon,Camden,1,01/22/2018,21:30,Pembina
2,"2430 2nd St n ,Minneapolis,MN",45.004469,-93.279508,1,5,Hawthorne,Near North,1,01/22/2018,21:35,Pembina
3,"4615 lyndale ave n,Minneapolis,MN",45.039000,-93.287739,1,4,Lind - Bohanon,Camden,1,01/22/2018,21:38,Pembina
4,"3520 Chicago ave s,Minneapolis,MN",44.938743,-93.262878,1,9,Central,Powderhorn,5,01/22/2018,21:43,Pembina


In [91]:
pembina.Community.unique()

array(['Camden', 'Near North', 'Powderhorn', 'Calhoun Isle', 'Central',
       'Northeast', 'University', 'Southwest', 'Nokomis', 'Longfellow',
       'Phillips'], dtype=object)

In [92]:
fix_neighborhood_names(pembina, neighborhoods)

55.  Old: Univeristy of Minnesota => New: University of Minnesota
336.  Old: CARAG => New: South Uptown
344.  Old: CARAG => New: South Uptown
376.  Old: CARAG => New: South Uptown
379.  Old: CARAG => New: South Uptown
399.  Old: CARAG => New: South Uptown
409.  Old: CARAG => New: South Uptown
424.  Old: Univeristy of Minnesota => New: University of Minnesota
437.  Old: CARAG => New: South Uptown
449.  Could not find neighborhood.  Old:  Marcy Holmes
450.  Old: CARAG => New: South Uptown
454.  Old: CARAG => New: South Uptown
460.  Old: Univeristy of Minnesota => New: University of Minnesota
479.  Old: CARAG => New: South Uptown
506.  Old: CARAG => New: South Uptown
511.  Old: CARAG => New: South Uptown
526.  Old: CARAG => New: South Uptown
528.  Old: CARAG => New: South Uptown
535.  Old: CARAG => New: South Uptown
537.  Old: CARAG => New: South Uptown
554.  Old: CARAG => New: South Uptown
578.  Old: CARAG => New: South Uptown
601.  Old: CARAG => New: South Uptown
619.  Old: CARAG => New

In [93]:
pembina.Neighborhood.isnull().sum()

0

In [94]:
pembina.Neighborhood.unique()

array(['McKinley', 'Lind - Bohanon', 'Hawthorne', 'Central', 'ECCO',
       'Loring Park', 'Sheridan', 'Marcy Holmes', 'Audubon Park',
       'Windom Park', 'Webber - Camden', 'Bryant', 'Powderhorn Park',
       'Columbia Park', 'Bancroft', "Steven's Square - Loring Heights",
       'Armatage', 'University of Minnesota', 'Regina',
       'Prospect Park - East River Road', 'Whittier', 'North Loop',
       'Holland', 'St. Anthony West', 'Seward', 'Cedar Riverside',
       'Corcoran', 'Near - North', 'Victory', 'Hiawatha',
       'Sumner - Glenwood', 'Wenonah', 'Keewaydin', 'Como', 'Jordan',
       'Folwell', 'Willard - Hay', 'Field', 'Downtown West',
       'Lowry Hill East', 'Elliot Park', 'Ericsson', 'Lyndale',
       'Linden Hills', 'Morris Park', 'King Field', 'Bottineau',
       'Standish', 'East Isles', 'St. Anthony East', 'West Calhoun',
       'Cedar - Isles - Dean', 'Lowry Hill', 'Northrop', 'South Uptown',
       'Logan Park', 'Northeast Park', 'Harrison', 'Longfellow',
       

In [95]:
pembina.to_csv(os.path.join(OUTPUT_DIR, "Pembina_Tows.csv"), index=False)

# Quincy 2019

In [98]:
quincy = load_csv(TOWING_FILES['Quincy'])
quincy.head()

,X,Y,OBJECTID,Service_Da,Address,Tow_Reason,Latitude,Longitude,Ward,Community,Neighborho,Tow_Zone,Day
0,-93.262331,44.943518,1,01/28/2019 09:22pm,"3241 CHICAGO AVE S,Minneapolis,MN",Snow Emergency,44.943518,-93.262331,9,Powderhorn,Powderhorn Park,6,1
1,-93.262914,44.942566,2,01/28/2019 09:33pm,"3318 CHICAGO AVE S,Minneapolis,MN",Snow Emergency,44.942566,-93.262914,9,Powderhorn,Central,5,1
2,-93.261746,44.942742,3,01/28/2019 09:36pm,"3320 CHICAGO AVE S,Minneapolis,MN",Snow Emergency,44.942742,-93.261746,9,Powderhorn,Powderhorn Park,6,1
3,-93.237383,45.031446,4,01/28/2019 09:42pm,"3500 Johnson ST NE ,Minneapolis,MN",Snow Emergency,45.031446,-93.237383,1,Northeast,Waite Park,2,1
4,-93.252653,44.941220,5,01/28/2019 09:46pm,"3400 blk of johnson,Minneapolis,MN",Snow Emergency,44.941220,-93.252653,9,Powderhorn,Powderhorn Park,6,1


In [99]:
quincy.rename(columns={'Neighborho': 'Neighborhood', 'Address': 'Location'}, inplace=True)

quincy['Date'] = quincy.Service_Da.map(get_date)
quincy['Time'] = quincy.Service_Da.map(get_time)
quincy.drop(columns=['X', 'Y', 'OBJECTID', 'Service_Da', 'Tow_Reason'], inplace=True)
quincy['Emergency'] = 'Quincy'

quincy.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Tow_Zone,Day,Date,Time,Emergency
0,"3241 CHICAGO AVE S,Minneapolis,MN",44.943518,-93.262331,9,Powderhorn,Powderhorn Park,6,1,01/28/2019,21:22,Quincy
1,"3318 CHICAGO AVE S,Minneapolis,MN",44.942566,-93.262914,9,Powderhorn,Central,5,1,01/28/2019,21:33,Quincy
2,"3320 CHICAGO AVE S,Minneapolis,MN",44.942742,-93.261746,9,Powderhorn,Powderhorn Park,6,1,01/28/2019,21:36,Quincy
3,"3500 Johnson ST NE ,Minneapolis,MN",45.031446,-93.237383,1,Northeast,Waite Park,2,1,01/28/2019,21:42,Quincy
4,"3400 blk of johnson,Minneapolis,MN",44.941220,-93.252653,9,Powderhorn,Powderhorn Park,6,1,01/28/2019,21:46,Quincy


In [100]:
quincy.Community.unique() # all ok

array(['Powderhorn', 'Northeast', 'Central', 'University', 'Camden',
       'Nokomis', 'Calhoun Isle', 'Near North', 'Phillips', 'Longfellow',
       'Southwest'], dtype=object)

In [101]:
fix_neighborhood_names(quincy, neighborhoods)

In [102]:
quincy.Neighborhood.unique()

array(['Powderhorn Park', 'Central', 'Waite Park', 'Loring Park',
       'Windom Park', 'Marcy Holmes', 'Bancroft', 'McKinley', 'Sheridan',
       'Webber - Camden', 'Elliot Park', 'Diamond Lake', 'Field',
       'Lind - Bohanon', 'Northrop', 'Audubon Park', 'Victory',
       'Camden Industrial', 'St. Anthony West', 'Lowry Hill East', 'Hale',
       'North Loop', 'Hawthorne', 'Harrison', 'East Phillips',
       'St. Anthony East', "Steven's Square - Loring Heights", 'Whittier',
       'Holland', 'Jordan', 'Bottineau', 'Willard - Hay', 'Downtown East',
       'Ericsson', 'Seward', 'Corcoran', 'University of Minnesota',
       'Bryn - Mawr', 'Cooper', 'Downtown West', 'Regina', 'Bryant',
       'Prospect Park - East River Road', 'Sumner - Glenwood', 'ECCO',
       'Hiawatha', 'Standish', 'Near - North', 'Logan Park',
       'Morris Park', 'Como', 'Folwell', 'Howe', 'Marshall Terrace',
       'Humboldt Industrial Area', 'Wenonah', 'Minnehaha', 'Longfellow',
       'King Field', 'Ventura V

In [103]:
quincy.to_csv(os.path.join(OUTPUT_DIR, "Quincy_Tows.csv"), index=False)

# Upton 2019

In [104]:
upton = load_csv(TOWING_FILES['Upton'])
upton.head()

,X,Y,OBJECTID,Call_Taken,Location,Latitude,Longitude,Ward,Community,Neighborho,Tow_Zone,Day,Snow_Emerg
0,-93.287689,45.009733,1,02/07/2019 09:26pm,"2900 LYNDALE AVE N,Minneapolis,MN",45.009733,-93.287689,5,Near North,Hawthorne,1,1,Upton
1,-93.287841,45.015031,2,02/07/2019 09:40pm,"3300-3399 Lyndale Ave N,Minneapolis,MN",45.015031,-93.287841,4,Camden,McKinley,1,1,Upton
2,-93.229661,44.988049,3,02/07/2019 09:46pm,"1531 COMO AVE SE,Minneapolis,MN",44.988049,-93.229661,1,University,Como,4,1,Upton
3,-93.288049,45.018490,4,02/07/2019 09:51pm,"3499 Lyndale Ave n,Minneapolis,MN",45.018490,-93.288049,4,Camden,McKinley,1,1,Upton
4,-93.287685,45.025889,5,02/07/2019 10:00pm,"3900 lyndale ave n ,Minneapolis,MN",45.025889,-93.287685,4,Camden,Webber - Camden,1,1,Upton


In [105]:
upton.rename(columns={'Neighborho': 'Neighborhood', 'Snow_Emerg': 'Emergency'}, inplace=True)

upton['Date'] = upton.Call_Taken.map(get_date)
upton['Time'] = upton.Call_Taken.map(get_time)
upton.drop(columns=['X', 'Y', 'OBJECTID', 'Call_Taken'], inplace=True)

upton.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Tow_Zone,Day,Emergency,Date,Time
0,"2900 LYNDALE AVE N,Minneapolis,MN",45.009733,-93.287689,5,Near North,Hawthorne,1,1,Upton,02/07/2019,21:26
1,"3300-3399 Lyndale Ave N,Minneapolis,MN",45.015031,-93.287841,4,Camden,McKinley,1,1,Upton,02/07/2019,21:40
2,"1531 COMO AVE SE,Minneapolis,MN",44.988049,-93.229661,1,University,Como,4,1,Upton,02/07/2019,21:46
3,"3499 Lyndale Ave n,Minneapolis,MN",45.018490,-93.288049,4,Camden,McKinley,1,1,Upton,02/07/2019,21:51
4,"3900 lyndale ave n ,Minneapolis,MN",45.025889,-93.287685,4,Camden,Webber - Camden,1,1,Upton,02/07/2019,22:00


In [106]:
upton.Community.unique()

array(['Near North', 'Camden', 'University', 'Powderhorn', 'Phillips',
       'Central', 'Nokomis', 'Northeast', 'Longfellow', 'Southwest',
       'Calhoun Isle'], dtype=object)

In [107]:
fix_neighborhood_names(upton, neighborhoods)

283.  Old: CARAG => New: South Uptown
467.  Old: CARAG => New: South Uptown
479.  Old: CARAG => New: South Uptown
499.  Old: CARAG => New: South Uptown


In [108]:
upton.to_csv(os.path.join(OUTPUT_DIR, "Upton_Tows.csv"), index=False)

# Westminster 2019

In [109]:
westminster = load_csv(TOWING_FILES['Westminster'])
westminster.head()

,X,Y,OBJECTID,Call_Taken,Location,Latitude,Longitude,Ward,Community,Neighborho,Tow_Zone,Day,Snow_Emerg
0,-93.288021,45.013111,1,02/12/2019 09:18pm,"3201-3299 lyndale ave n ,Minneapolis,MN",45.013111,-93.288021,4,Camden,McKinley,1,1,Westminster
1,-93.262588,44.941390,2,02/12/2019 09:19pm,"3398 Chicago ave s,Minneapolis,MN",44.941390,-93.262588,9,Powderhorn,Central,5,1,Westminster
2,-93.262606,44.943159,3,02/12/2019 09:19pm,"3301 Chicago Avenue South,Minneapolis,MN",44.943159,-93.262606,9,Powderhorn,Central,5,1,Westminster
3,-93.288036,45.016708,4,02/12/2019 09:26pm,"3400-3498 lyndale ave n ,Minneapolis,MN",45.016708,-93.288036,4,Camden,McKinley,1,1,Westminster
4,-93.288049,45.018490,5,02/12/2019 09:34pm,"3498 Lyndale Ave n,Minneapolis,MN",45.018490,-93.288049,4,Camden,McKinley,1,1,Westminster


In [110]:
westminster.rename(columns={'Neighborho': 'Neighborhood', 'Snow_Emerg': 'Emergency'}, inplace=True)

westminster['Date'] = westminster.Call_Taken.map(get_date)
westminster['Time'] = westminster.Call_Taken.map(get_time)
westminster.drop(columns=['X', 'Y', 'OBJECTID', 'Call_Taken'], inplace=True)

westminster.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Tow_Zone,Day,Emergency,Date,Time
0,"3201-3299 lyndale ave n ,Minneapolis,MN",45.013111,-93.288021,4,Camden,McKinley,1,1,Westminster,02/12/2019,21:18
1,"3398 Chicago ave s,Minneapolis,MN",44.941390,-93.262588,9,Powderhorn,Central,5,1,Westminster,02/12/2019,21:19
2,"3301 Chicago Avenue South,Minneapolis,MN",44.943159,-93.262606,9,Powderhorn,Central,5,1,Westminster,02/12/2019,21:19
3,"3400-3498 lyndale ave n ,Minneapolis,MN",45.016708,-93.288036,4,Camden,McKinley,1,1,Westminster,02/12/2019,21:26
4,"3498 Lyndale Ave n,Minneapolis,MN",45.018490,-93.288049,4,Camden,McKinley,1,1,Westminster,02/12/2019,21:34


In [111]:
westminster.Community.unique()

array(['Camden', 'Powderhorn', 'Central', 'University', 'Nokomis',
       'Near North', 'Northeast', 'Phillips', 'Longfellow',
       'Calhoun Isle', 'Southwest'], dtype=object)

In [112]:
fix_neighborhood_names(westminster, neighborhoods)

156.  Old: CARAG => New: South Uptown
301.  Old: CARAG => New: South Uptown
451.  Old: CARAG => New: South Uptown
454.  Old: CARAG => New: South Uptown
462.  Old: CARAG => New: South Uptown
518.  Old: CARAG => New: South Uptown
521.  Old: CARAG => New: South Uptown
546.  Old: CARAG => New: South Uptown
571.  Old: CARAG => New: South Uptown
599.  Old: CARAG => New: South Uptown
601.  Old: CARAG => New: South Uptown
608.  Old: CARAG => New: South Uptown
616.  Old: CARAG => New: South Uptown
639.  Old: CARAG => New: South Uptown
679.  Old: CARAG => New: South Uptown
756.  Old: CARAG => New: South Uptown
767.  Old: CARAG => New: South Uptown
776.  Old: CARAG => New: South Uptown
796.  Old: CARAG => New: South Uptown
799.  Old: CARAG => New: South Uptown
814.  Old: CARAG => New: South Uptown
821.  Old: CARAG => New: South Uptown
829.  Old: CARAG => New: South Uptown
838.  Old: CARAG => New: South Uptown
871.  Old: CARAG => New: South Uptown
873.  Old: CARAG => New: South Uptown
896.  Old: C

In [113]:
westminster.to_csv(os.path.join(OUTPUT_DIR, "Westminster_Tows.csv"), index=False)

# Xerxes 2018

In [117]:
xerxes = load_csv(TOWING_FILES['Xerxes'])
xerxes.head()

,X,Y,FID,Call_Taken,Customer,Location,Latitude,Longitude,Snow_Emerg,Day,Ward,Neighborho,Community,Tow_Dist
0,-93.249046,44.984485,1,02/24/2018 12:20am,Snow Emergency,"299 5th ave se,Minneapolis,MN",44.984485,-93.249046,Xerxes,1,3,Marcy Holmes,University,4
1,-93.262311,44.947508,2,02/23/2018 10:30pm,Snow Emergency,"3025 Chicago Ave S,Minneapolis,MN",44.947508,-93.262311,Xerxes,1,9,Powderhorn Park,Powderhorn,6
2,-93.241708,44.986318,3,02/24/2018 12:20pm,Snow Emergency,"620 8th ave se,Minneapolis,MN",44.986318,-93.241708,Xerxes,2,3,Marcy Holmes,University,4
3,-93.262193,44.927397,4,02/23/2018 11:10pm,Snow Emergency,"4145 CHICAGO AVE S,Minneapolis,MN",44.927397,-93.262193,Xerxes,1,8,Bancroft,Powderhorn,6
4,-93.262498,44.927287,5,02/23/2018 11:11pm,Snow Emergency,"4146 Chicago Ave S,Minneapolis,MN",44.927287,-93.262498,Xerxes,1,8,Bancroft,Powderhorn,6


In [118]:
xerxes.rename(columns={'Neighborho': 'Neighborhood', 'Snow_Emerg': 'Emergency', 'Tow_Dist': 'Tow_Zone'}, inplace=True)

xerxes['Date'] = xerxes.Call_Taken.map(get_date)
xerxes['Time'] = xerxes.Call_Taken.map(get_time)
xerxes.drop(columns=['X', 'Y', 'FID', 'Call_Taken', 'Customer'], inplace=True)

xerxes.head()

,Location,Latitude,Longitude,Emergency,Day,Ward,Neighborhood,Community,Tow_Zone,Date,Time
0,"299 5th ave se,Minneapolis,MN",44.984485,-93.249046,Xerxes,1,3,Marcy Holmes,University,4,02/24/2018,00:20
1,"3025 Chicago Ave S,Minneapolis,MN",44.947508,-93.262311,Xerxes,1,9,Powderhorn Park,Powderhorn,6,02/23/2018,22:30
2,"620 8th ave se,Minneapolis,MN",44.986318,-93.241708,Xerxes,2,3,Marcy Holmes,University,4,02/24/2018,12:20
3,"4145 CHICAGO AVE S,Minneapolis,MN",44.927397,-93.262193,Xerxes,1,8,Bancroft,Powderhorn,6,02/23/2018,23:10
4,"4146 Chicago Ave S,Minneapolis,MN",44.927287,-93.262498,Xerxes,1,8,Bancroft,Powderhorn,6,02/23/2018,23:11


In [119]:
xerxes.Community.unique()

array(['University', 'Powderhorn', 'Nokomis', 'Longfellow', 'Southwest',
       'Calhoun Isle', 'Central', 'Near North', 'Phillips', 'Northeast',
       'Camden'], dtype=object)

In [120]:
fix_neighborhood_names(xerxes, neighborhoods)

222.  Old: CARAG => New: South Uptown
224.  Old: CARAG => New: South Uptown
225.  Old: CARAG => New: South Uptown
228.  Old: CARAG => New: South Uptown
229.  Old: CARAG => New: South Uptown
232.  Old: CARAG => New: South Uptown
233.  Old: CARAG => New: South Uptown
234.  Old: CARAG => New: South Uptown
235.  Old: CARAG => New: South Uptown
236.  Old: CARAG => New: South Uptown
271.  Old: CARAG => New: South Uptown
283.  Old: CARAG => New: South Uptown
285.  Old: CARAG => New: South Uptown
286.  Old: CARAG => New: South Uptown
287.  Old: CARAG => New: South Uptown
288.  Old: CARAG => New: South Uptown
289.  Old: CARAG => New: South Uptown
299.  Old: CARAG => New: South Uptown
303.  Old: CARAG => New: South Uptown
305.  Old: CARAG => New: South Uptown


In [121]:
xerxes.to_csv(os.path.join(OUTPUT_DIR, "Xerxes_Tows.csv"), index=False)

# Yale 2019

In [124]:
yale = load_csv(TOWING_FILES['Yale'])
yale.head()

,X,Y,OBJECTID,Call_Taken,Location,Tow_Reason,Latitude,Longitude,Ward,Community,Neighborho,Tow_Zone,Day,Snow_Emerg
0,-93.287718,45.038700,1,02/20/2019 09:17pm,"4601 Lyndale Ave n ,Minneapolis,MN",Snow Emergency,45.038700,-93.287718,4,Camden,Lind - Bohanon,1,1,Yale
1,-93.287345,45.040554,2,02/20/2019 09:21pm,"4699 Lyndale Ave n,Minneapolis,MN",Snow Emergency,45.040554,-93.287345,4,Camden,Lind - Bohanon,1,1,Yale
2,-93.235449,44.987643,3,02/20/2019 09:23pm,"1100 COMO AVE SE,Minneapolis,MN",Snow Emergency,44.987643,-93.235449,2,University,Como,4,1,Yale
3,-93.262689,44.948357,4,02/20/2019 09:25pm,"3000 Chicago ave ,Minneapolis,MN",Snow Emergency,44.948357,-93.262689,6,Phillips,Phillips West,3,1,Yale
4,-93.288451,45.027751,5,02/20/2019 09:33pm,"4001 LYNDALE AVE N ,Minneapolis,MN",Snow Emergency,45.027751,-93.288451,4,Camden,Webber - Camden,1,1,Yale


In [125]:
yale.rename(columns={'Neighborho': 'Neighborhood', 'Snow_Emerg': 'Emergency'}, inplace=True)

yale['Date'] = yale.Call_Taken.map(get_date)
yale['Time'] = yale.Call_Taken.map(get_time)
yale.drop(columns=['X', 'Y', 'OBJECTID', 'Call_Taken', 'Tow_Reason'], inplace=True)

yale.head()

,Location,Latitude,Longitude,Ward,Community,Neighborhood,Tow_Zone,Day,Emergency,Date,Time
0,"4601 Lyndale Ave n ,Minneapolis,MN",45.038700,-93.287718,4,Camden,Lind - Bohanon,1,1,Yale,02/20/2019,21:17
1,"4699 Lyndale Ave n,Minneapolis,MN",45.040554,-93.287345,4,Camden,Lind - Bohanon,1,1,Yale,02/20/2019,21:21
2,"1100 COMO AVE SE,Minneapolis,MN",44.987643,-93.235449,2,University,Como,4,1,Yale,02/20/2019,21:23
3,"3000 Chicago ave ,Minneapolis,MN",44.948357,-93.262689,6,Phillips,Phillips West,3,1,Yale,02/20/2019,21:25
4,"4001 LYNDALE AVE N ,Minneapolis,MN",45.027751,-93.288451,4,Camden,Webber - Camden,1,1,Yale,02/20/2019,21:33


In [126]:
yale.Community.unique()

array(['Camden', 'University', 'Phillips', 'Powderhorn', 'Near North',
       'Central', 'Northeast', 'Calhoun Isle', 'Nokomis', 'Longfellow',
       'Southwest'], dtype=object)

In [127]:
fix_neighborhood_names(yale, neighborhoods)

201.  Old: CARAG => New: South Uptown
211.  Old: CARAG => New: South Uptown
337.  Old: CARAG => New: South Uptown
363.  Old: CARAG => New: South Uptown
400.  Old: CARAG => New: South Uptown
413.  Old: CARAG => New: South Uptown
436.  Old: CARAG => New: South Uptown
461.  Old: CARAG => New: South Uptown
465.  Old: CARAG => New: South Uptown
467.  Old: CARAG => New: South Uptown
469.  Old: CARAG => New: South Uptown
483.  Old: CARAG => New: South Uptown
499.  Old: CARAG => New: South Uptown
523.  Old: CARAG => New: South Uptown
524.  Old: CARAG => New: South Uptown
526.  Old: CARAG => New: South Uptown
527.  Old: CARAG => New: South Uptown
548.  Old: CARAG => New: South Uptown
570.  Old: CARAG => New: South Uptown
585.  Old: CARAG => New: South Uptown
587.  Old: CARAG => New: South Uptown
634.  Old: CARAG => New: South Uptown
640.  Old: CARAG => New: South Uptown
665.  Old: CARAG => New: South Uptown
674.  Old: CARAG => New: South Uptown
704.  Old: CARAG => New: South Uptown
707.  Old: C

In [128]:
yale.to_csv(os.path.join(OUTPUT_DIR, "Yale_Tows.csv"), index=False)

# Yardville 2018

In [129]:
yardville = load_csv(TOWING_FILES['Yardville'])
yardville.head()

,X,Y,FID,Call_Taken,Customer,Location,Latitude,Longitude,Snow_Emerg,Day,Ward,Neighborho,Community,Tow_Dist
0,-93.262490,44.933471,1,02/25/2018 10:51pm,Snow Emergency,"3820 CHICAGO AVE S,Minneapolis,MN",44.933471,-93.262490,Yardville,1,8,Bancroft,Powderhorn,6
1,-93.237405,45.021521,2,02/25/2018 09:58pm,Snow Emergency,"2936 Johnson St NE,Minneapolis,MN",45.021521,-93.237405,Yardville,1,1,Audubon Park,Northeast,2
2,-93.261285,44.938653,3,02/26/2018 01:03pm,Snow Emergency,"3526 Elliott Avenue,Minneapolis,MN",44.938653,-93.261285,Yardville,2,9,Powderhorn Park,Powderhorn,6
3,-93.269084,45.000589,4,02/25/2018 10:02pm,Snow Emergency,"119 13th Ave NE,Minneapolis,MN",45.000589,-93.269084,Yardville,1,3,Sheridan,Northeast,2
4,-93.248997,44.944920,5,02/26/2018 01:04pm,Snow Emergency,"3148 18th ave so,Minneapolis,MN",44.944920,-93.248997,Yardville,2,9,Powderhorn Park,Powderhorn,6


In [130]:
yardville.rename(columns={'Neighborho': 'Neighborhood', 'Snow_Emerg': 'Emergency', 'Tow_Dist': 'Tow_Zone'}, inplace=True)

yardville['Date'] = yardville.Call_Taken.map(get_date)
yardville['Time'] = yardville.Call_Taken.map(get_time)
yardville.drop(columns=['X', 'Y', 'FID', 'Call_Taken', 'Customer'], inplace=True)

yardville.head()

,Location,Latitude,Longitude,Emergency,Day,Ward,Neighborhood,Community,Tow_Zone,Date,Time
0,"3820 CHICAGO AVE S,Minneapolis,MN",44.933471,-93.262490,Yardville,1,8,Bancroft,Powderhorn,6,02/25/2018,22:51
1,"2936 Johnson St NE,Minneapolis,MN",45.021521,-93.237405,Yardville,1,1,Audubon Park,Northeast,2,02/25/2018,21:58
2,"3526 Elliott Avenue,Minneapolis,MN",44.938653,-93.261285,Yardville,2,9,Powderhorn Park,Powderhorn,6,02/26/2018,13:03
3,"119 13th Ave NE,Minneapolis,MN",45.000589,-93.269084,Yardville,1,3,Sheridan,Northeast,2,02/25/2018,22:02
4,"3148 18th ave so,Minneapolis,MN",44.944920,-93.248997,Yardville,2,9,Powderhorn Park,Powderhorn,6,02/26/2018,13:04


In [131]:
yardville.Community.unique()

array(['Powderhorn', 'Northeast', 'Nokomis', 'Longfellow', 'Calhoun Isle',
       'Southwest', 'University', 'Phillips', 'Central', 'Near North',
       'Camden'], dtype=object)

In [132]:
fix_neighborhood_names(yardville, neighborhoods)

140.  Old: CARAG => New: South Uptown
143.  Old: CARAG => New: South Uptown
144.  Old: CARAG => New: South Uptown
145.  Old: CARAG => New: South Uptown
146.  Old: CARAG => New: South Uptown
150.  Old: CARAG => New: South Uptown
152.  Old: CARAG => New: South Uptown
156.  Old: CARAG => New: South Uptown
157.  Old: CARAG => New: South Uptown
185.  Old: CARAG => New: South Uptown
187.  Old: CARAG => New: South Uptown
188.  Old: CARAG => New: South Uptown
189.  Old: CARAG => New: South Uptown
190.  Old: CARAG => New: South Uptown
233.  Could not find neighborhood.  Old:  Prospect Park - East River Road
537.  Could not find neighborhood.  Old:  Columbia Park


In [133]:
yardville.Neighborhood.isnull().sum()

0

In [134]:
yardville.to_csv(os.path.join(OUTPUT_DIR, "Yardville_Tows.csv"), index=False)

In [150]:
# Can we read in all 16 towing files and produce a single file:

# First add the two missing emergencies that we converted from GeoJSONs:
TOWING_FILES['Grant'] = ""
TOWING_FILES['Polk'] = ""

def review_and_concat_files(towing_files=TOWING_FILES, output_dir=OUTPUT_DIR):
    
    dfs = []
    
    for key in towing_files.keys():
        filename = key + "_Tows.csv"
        filepath = os.path.join(output_dir, filename)
        tmp = pd.read_csv(filepath)
        print(tmp.columns)
        print(tmp.shape)
        dfs.append(tmp)
        
    all_dfs = pd.concat(dfs)
    print(all_dfs.columns)
    print(all_dfs.shape)
    all_dfs.to_csv(os.path.join(OUTPUT_DIR, "All_Tows.csv"), index=False)
    
    return all_dfs

In [151]:
concat_dfs = review_and_concat_files()

Index(['Location', 'Latitude', 'Longitude', 'Ward', 'Community',
       'Neighborhood', 'Tow_Zone', 'Day', 'Emergency', 'Date', 'Time'],
      dtype='object')
(948, 11)
Index(['Location', 'Latitude', 'Longitude', 'Ward', 'Neighborhood',
       'Community', 'Day', 'Tow_Zone', 'Date', 'Time', 'Emergency'],
      dtype='object')
(1354, 11)
Index(['Location', 'Latitude', 'Longitude', 'Ward', 'Community',
       'Neighborhood', 'Tow_Zone', 'Day', 'Emergency', 'Date', 'Time'],
      dtype='object')
(982, 11)
Index(['Location', 'Latitude', 'Longitude', 'Ward', 'Neighborhood',
       'Community', 'Day', 'Tow_Zone', 'Date', 'Time', 'Emergency'],
      dtype='object')
(1328, 11)
Index(['Location', 'Latitude', 'Longitude', 'Ward', 'Community',
       'Neighborhood', 'Tow_Zone', 'Day', 'Emergency', 'Date', 'Time'],
      dtype='object')
(844, 11)
Index(['Location', 'Latitude', 'Longitude', 'Ward', 'Neighborhood',
       'Community', 'Day', 'Tow_Zone', 'Date', 'Time', 'Emergency'],
      dtype='obj

C:\Users\dadra\anaconda\Anaconda3\envs\PythonWebMongo\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [152]:
concat_dfs.shape

(17464, 11)

In [153]:
concat_dfs.groupby(['Emergency'])['Location'].count()

Emergency
Armatage         948
Dana            1354
Diamond Lake     982
Ferry           1328
Grant           1736
Howe             844
Jane            1430
Olive           1161
Pembina          962
Polk            1480
Quincy           617
Upton            805
Westminster      980
Xerxes          1215
Yale             894
Yardville        728
Name: Location, dtype: int64